In [57]:
# Spec: Core i7 8550U 4 cores 8 threads
# Jupyter notebook on windows 11
# python 3.9.7

# DATASET
# https://github.com/datquocnguyen/VnDT 1400 sentences

# REQUIREMENTS
# conll==4.4.1
# ViSpacy==0.0.1 https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz
# Underthesea==1.3.4a0
# Vncorenlp==1.0.3 
# Java


In [58]:
from conllu import parse
import time

In [59]:

# Format for data [[[word, pos, entity], [word, pos, entity],...]]
with open ('data/vi_vtb-ud-train.conllu', 'r', encoding='utf-8') as f:
	text = f.read()

sentences = parse(text)

text = ''
sents = []
groundtruth = []
for tokenlist in sentences:
    tagged = []
    sent = tokenlist.metadata['text']
    for item in tokenlist:
        tagged.append([item['form'],item['xpos'],' '])
    text += sent + ' '
    groundtruth.append(tagged)
    sents.append(sent)

print(f"Num sentences: {len(sents)}")
print(f"Sentence: {sents[0]}\nTag: {groundtruth[0]}")

Num sentences: 1400
Sentence: mảnh đất của đạn bom không còn người nghèo.
Tag: [['mảnh', 'Nc', ' '], ['đất', 'N', ' '], ['của', 'E', ' '], ['đạn', 'N', ' '], ['bom', 'N', ' '], ['không', 'R', ' '], ['còn', 'V', ' '], ['người', 'N', ' '], ['nghèo', 'A', ' '], ['.', '.', ' ']]


In [60]:
sentences

[TokenList<mảnh, đất, của, đạn, bom, không, còn, người, nghèo, .>,
 TokenList<chiến tranh, đi, qua, để, lại, quê hương, Thái Mỹ, ,, huyện, Củ Chi, LBKT, TP. HCM, RBKT, hộ, gia đình, chính sách, và, hơn, 2, ha, ", đất, thép, ", .>,
 TokenList<buông, súng, ,, người, Thái Mỹ, cố, quên, những, mất mát, cầm, lấy, cây, cuốc, .>,
 TokenList<Nhưng, mảnh, đất, thấm, máu, của, gần, 500, liệt sĩ, và, đầy, mảnh, bom đạn, ấy, không, đủ, cái, ăn, ,, cái, mặc, .>,
 TokenList<Lại, quăng, cuốc, ,, gần, 90, %, gia đình, phải, đeo, giỏ, đi, buôn lậu, .>,
 TokenList<Nhưng, đó, là, chuyện, của, Thái Mỹ, 20, năm, về, trước, ,, còn, bây giờ, ...>,
 TokenList<đất, nghèo, trở mình, .>,
 TokenList<", chỗ, này, xưa, là, đồn bót, nè, @>,
 TokenList<Kia, là, những, ngôi, nhà, vách, đất, .>,
 TokenList<Cũng, bằng, hai, cùi tay, lạ đời, ,, anh, lái, xe máy, băng băng, trên, con, đường, phẳng lì, ...>,
 TokenList<", Tất cả, đường, bêtông, nội đồng, là, thành quả, nhà nước, và, nhân dân, cùng, làm, ", -, anh, tự hào, 

In [61]:
class Spacy_tokenize:
 
	def __init__(self):
		import spacy
		self.nlp = spacy.load('vi_core_news_lg')


	def tokenize(self,text):
		output = []
		doc = self.nlp(text)

		for token in doc:
		    output.append([token.text,token.tag_,''])
		    #print(token.text, token.lemma_, token.tag_, token.pos_, token.dep_,
		    #        token.shape_, token.is_alpha, token.is_stop)

		return output

	def info(self):
		return('ViSpacy')
	
	def close(self):
		pass

In [62]:
class Underthesea_tokenize:
	def __init__(self):
		from underthesea import word_tokenize
		from underthesea import ner
		self.ner = ner
	def tokenize(self,text):
		output = []
		ners = self.ner(text)
		for item in ners:
			output.append([item[0],item[1],item[3]])
		
		return output

	def info(self):
		return('Underthesea')
		
	def close(self):
		pass

In [63]:
class VncoreNLP_tokenize:
	# To perform word segmentation, POS tagging, NER and then dependency parsing
	# annotator = VnCoreNLP("VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')
	def __init__(self):
		from vncorenlp import VnCoreNLP
		self.annotator = VnCoreNLP("VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')
		def tokenize(self,text):
			output = []
			annotated_text = self.annotator.annotate(text)
			for sent in annotated_text['sentences']:
				for item in sent:
					output.append([item['form'].replace('_',' '),item['posTag'],item['nerLabel']])
			return output
			
	def info(self):
		return('VnCoreNLP')

	def close(self):
		self.annotator.close()
	

In [64]:
for t in (Spacy_tokenize,Underthesea_tokenize,VncoreNLP_tokenize):
    t = t()
    count = 0
    wordcount = 0
    poscount = 0
    sercount = 0


    time_sum = 0
    index = 0

    for sent in sents:

        start = time.time()
        predict = t.tokenize(sent)
        time_sum += time.time() - start
        count += len(groundtruth[index])

        if len(predict) == len(groundtruth[index]): # only add to count if num predicted words equals that of groundtruth
            for item,gt in zip(predict,groundtruth[index]):  # item = [word, pos, entity]

                #print(item,gt)
                if item[0] == gt[0]:
                    wordcount += 1
                if item[1] == gt[1]:
                    poscount += 1
                if item[2] == gt[2]:
                    sercount += 1
        index += 1

    # Corrected segmented word and entity / total word count
    wordsegacc = wordcount/count
    posacc = poscount/count
    seracc = sercount/count

    print()
    print(t.info())
    print(f"Tagging time: {time_sum:.5f} Word segmentation acc: {wordsegacc:.5f} Pos tag acc: {posacc:.5f} ")


    t.close()


ViSpacy
Tagging time: 19.60007 Word segmentation acc: 0.55272 Pos tag acc: 0.69682 

Underthesea
Tagging time: 15.93241 Word segmentation acc: 0.80813 Pos tag acc: 0.60937 


AttributeError: 'VncoreNLP_tokenize' object has no attribute 'tokenize'

In [ ]:
from CocCocTokenizer import PyTokenizer

# load_nontone_data is True by default
T = PyTokenizer(load_nontone_data=True)

# tokenize_option:
# 	0: TOKENIZE_NORMAL (default)
#	1: TOKENIZE_HOST
#	2: TOKENIZE_URL
print(T.word_tokenize("xin chào, tôi là người Việt Nam", tokenize_option=0))

# output: ['xin', 'chào', ',', 'tôi', 'là', 'người', 'Việt_Nam']

ModuleNotFoundError: No module named 'CocCocTokenizer'